# This is a `basicRaytracing` python-based example of CPU 3D ra-tracing rendering application
## This notebook has been updated by Prof. George Papagiannakis as an introduction to CPU-based computer graphics Elements 
### the code is based on the book "Ray Tracing in One Weekend" by Peter Shirley (https://raytracing.github.io/books/RayTracingInOneWeekend.html)
---


## 1. Overview

The code is based on python libraries and is a basic example of raytracing. 



## 2. Output an image
The code below outputs a simple background image using the PPM file format. The PPM file format is a very simple file format that can be used to output an image. The code below outputs a PPM image file that is `image_width` pixels wide and `image_height` pixels high. The image is a gradient that goes from black to *red* on top right, *green* to bottom left and the combination of *red + green = yellow* on bottom right.

In [1]:
import numpy as np 

# Set image dimensions
image_width = 256
image_height = 256

# Open a file to write the PPM image
with open("output.ppm", "w") as file:
    # Write PPM header
    file.write("P3\n")
    file.write(f"{image_width} {image_height}\n")
    file.write("255\n")

    # Loop over each pixel and calculate color values
    for j in range(image_height):
        # add a progress indicator
        print(f"\rScanlines remaining: {image_height - j}", end=' ', flush=True)
        for i in range(image_width):
            r = i / (image_width - 1)
            g = j / (image_height - 1)
            b = 0.0

            # Convert color values to integers
            ir = int(255.999 * r)
            ig = int(255.999 * g)
            ib = int(255.999 * b)

            # Write the pixel color values to the file
            file.write(f"{ir} {ig} {ib}\n")

Scanlines remaining: 1      

This Python script performs the following operations:

- It sets the image `image_width` and `image_height`.
- It opens a new file `output.ppm`` to write the image data.
- It writes the header for the PPM file format.
- It then iterates over each pixel in the image, calculating the *red (r)* and *green (g)* components based on the pixel's position. The *blue* component *(b)* is set to 0.
- It **converts** these *floating-point* color values to **integers** in the **range 0 to 255**.
- Finally, it writes these color values into the file *for each pixel*.
-
When this script is run, it will create a file named `output.ppm`` which will contain the image data in the PPM format, producing a gradient image.

The conversion of the color values to integers between 0 and 255 in the given code is a crucial step for representing colors in digital images, especially in formats like PPM (Portable Pixmap). Here's an explanation of how this conversion works:

1. **Fractional Color Values**: Initially, the color values (`r`, `g`, and `b`) are calculated as fractions. These fractions represent how "intense" or "bright" the color is. A value of 0 means no intensity (black), and a value close to 1 means full intensity. For example, in the code, `r` and `g` values vary from 0 to almost 1 across the width and height of the image.

2. **Scaling to 0-255 Range**: Most digital images use 8 bits per color channel, which allows for 256 different levels of intensity per channel (from 0 to 255). To convert a fractional intensity value to this range, it is multiplied by the maximum intensity level (255). However, to ensure the maximum fractional value (just below 1) maps correctly to 255, the multiplier is slightly less than 256, often chosen as 255.999. This adjustment helps in rounding off the values correctly.

3. **Conversion to Integer**: 
   - After multiplication, the resulting value is in the range from 0 to 255.999.
   - This value is then converted to an integer. In C++, the `static_cast<int>()` is used, and in Python, this is done simply by using `int()`. This step truncates (or effectively rounds down) the decimal part, resulting in an integer value in the desired range of 0 to 255.

4. **Application in the Code**: 
   - In the context of the code, each pixel's color intensity for red (`r`) and green (`g`) is calculated based on its position.
   - `r` varies horizontally, and `g` varies vertically. The blue component (`b`) is set to 0, which means there will be no blue intensity in any pixel.
   - These values are then scaled and converted to integers, giving us the appropriate red and green color values for each pixel to be written in the PPM file.

5. **Print progress indicator**:
   - This Python code uses f-string formatting to embed the expression `image_height - j` within the string. The `end=' '` argument replaces the default newline character that `print` adds to the end of its output with a space. The `flush=True` argument ensures that the output is flushed (i.e., immediately written) to the terminal, which is the equivalent of `std::flush` in C++.

By performing this conversion for each pixel, the code effectively generates the color information for an entire image, where each pixel's color is represented by a set of three integers (representing the red, green, and blue channels) in the range of 0 to 255.

> As described, the image shows a gradient:

- The **red component (R)** increases from *left to right*.
- The **green component (G)** increases from *top to bottom*.
- The **blue component (B)** is consistently zero across the image.

This results in a color transition from black at the top-left corner (where both R and G are zero) to yellow at the bottom-right corner (where both R and G are at their maximum, with no blue component).

In [ ]:
# use the PIL library to display the image we just created
from PIL import Image

# Load the image from a file
image = Image.open("output.ppm")
# Display the image
display(image)

## 3. The vec3 class

We need a basic vec3 type like the one in the C++ book. Following Python traditions, class members are not encaspsulated with getters/setters (if needed, you can always turn them into properties later, without breaking backward compatibility). We also add a few useful methods, that will allow us to use the Vec3 almost as if it was a regular numpy array.

Almost all graphics programs have some class(es) for storing geometric vectors and colors. In many systems thesevectors are 4D (3D position plus a homogeneous coordinate for geometry, or RGB plus an alpha transparencycomponent for colors). For our purposes, three coordinates suffice. We’ll use the same class vec3 for colors,locations, directions, offsets, whatever. Some people don’t like this because it doesn’t prevent you from doingsomething silly, like subtracting a position from a color. They have a good point, but we’re going to always take the“less code” route when not obviously wrong. In spite of this, we do declare two aliases for vec3: point3 and color.Since these two types are just aliases for vec3, you won't get warnings if you pass a color to a function expecting apoint3, and nothing is stopping you from adding a point3 to a color, but it makes the code a little bit easier to readand to understand.

In [ ]:
""""
class Vec3:
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = np.array(x, dtype=np.float32)
        self.y = np.array(y, dtype=np.float32)
        self.z = np.array(z, dtype=np.float32)

    @staticmethod
    def empty(size):
        x = np.empty(size, dtype=np.float32)
        y = np.empty(size, dtype=np.float32)
        z = np.empty(size, dtype=np.float32)
        return Vec3(x,y,z)

    @staticmethod
    def zeros(size):
        x = np.zeros(size, dtype=np.float32)
        y = np.zeros(size, dtype=np.float32)
        z = np.zeros(size, dtype=np.float32)
        return Vec3(x,y,z)

    @staticmethod
    def ones(size):
        x = np.ones(size, dtype=np.float32)
        y = np.ones(size, dtype=np.float32)
        z = np.ones(size, dtype=np.float32)
        return Vec3(x,y,z)
    
    @staticmethod
    def where(condition, v1, v2):
        x = np.where(condition, v1.x, v2.x)
        y = np.where(condition, v1.y, v2.y)
        z = np.where(condition, v1.z, v2.z)
        return Vec3(x,y,z)
    
    def clip(self, vmin, vmax):
        x = np.clip(self.x, vmin, vmax)
        y = np.clip(self.y, vmin, vmax)
        z = np.clip(self.z, vmin, vmax)
        return Vec3(x,y,z)

    def fill(self, value):
        self.x.fill(value)
        self.y.fill(value)
        self.z.fill(value)

    def repeat(self, n):
        x = np.repeat(self.x, n)
        y = np.repeat(self.y, n)
        z = np.repeat(self.z, n)
        return Vec3(x,y,z)
    
    def __str__(self):
        return 'vec3: x:%s y:%s z:%s' % (str(self.x), str(self.y), str(self.z))
    
    def __len__(self):
        return self.x.size

    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)

    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)

    def __mul__(self, scalar):
        return Vec3(self.x*scalar, self.y*scalar, self.z*scalar)

    def multiply(self, other):
        return Vec3(self.x * other.x, self.y * other.y, self.z * other.z)

    def __truediv__(self, scalar):
        return Vec3(self.x/scalar, self.y/scalar, self.z/scalar)
    
    def tile(self, shape):
        '''Replicate np.tile on each component'''
        return Vec3(np.tile(self.x, shape), np.tile(self.y, shape), np.tile(self.z, shape))

    def __getitem__(self, idx):
        '''Extract a vector subset'''
        return Vec3(self.x[idx], self.y[idx], self.z[idx])
    
    def __setitem__(self, idx, other):
        '''Set a vector subset from another vector'''
        self.x[idx] = other.x
        self.y[idx] = other.y
        self.z[idx] = other.z

    def join(self):
        '''Join the three components into a single 3xN array'''
        return np.vstack((self.x, self.y, self.z))
    
    def append(self, other):
        '''Append another vector to this one.
        Use concatenate() because cupy has no append function.
        '''
        self.x = np.concatenate((self.x, other.x))
        self.y = np.concatenate((self.y, other.y))
        self.z = np.concatenate((self.z, other.z))
        

## Aliases
Point3 = Vec3
Color = Vec3

## Utility functions
def unit_vector(v):
    return v / length(v)

def dot(a, b):
    return a.x*b.x + a.y*b.y + a.z*b.z

def length(v):
    return length_squared(v)**0.5

def length_squared(v):
    return v.x*v.x + v.y*v.y + v.z*v.z

def cross(a, b):
    return Vec3(a.y*b.z - a.z*b.y,
                -(a.x*b.z - a.z*b.x),
                a.x*b.y - a.y*b.x)
"""

In [8]:
import numpy as np

class Vec3:
    def __init__(self, e0=0, e1=0, e2=0):
        self.e = np.array([e0, e1, e2])

    def x(self):
        return self.e[0]

    def y(self):
        return self.e[1]

    def z(self):
        return self.e[2]

    def __neg__(self):
        return Vec3(-self.e[0], -self.e[1], -self.e[2])

    def __getitem__(self, i):
        return self.e[i]

    def __setitem__(self, i, value):
        self.e[i] = value

    def __iadd__(self, other):
        self.e += other.e
        return self

    def __imul__(self, t):
        self.e *= t
        return self

    def __itruediv__(self, t):
        self.e /= t
        return self
    
    def __mul__(self, scalar):
        return Vec3(self.x*scalar, self.y*scalar, self.z*scalar)

    def length(self):
        return np.linalg.norm(self.e)

    def length_squared(self):
        return np.dot(self.e, self.e)

    def near_zero(self):
        s = 1e-8
        return (np.abs(self.e[0]) < s) and (np.abs(self.e[1]) < s) and (np.abs(self.e[2]) < s)

    @staticmethod
    def random():
        return Vec3(np.random.random(), np.random.random(), np.random.random())

    @staticmethod
    def random_range(min, max):
        return Vec3(np.random.uniform(min, max), np.random.uniform(min, max), np.random.uniform(min, max))

# Utility Functions
def random_in_unit_disk():
    while True:
        p = Vec3(np.random.uniform(-1,1), np.random.uniform(-1,1), 0)
        if p.length_squared() < 1:
            return p

def random_in_unit_sphere():
    while True:
        p = Vec3.random_range(-1, 1)
        if p.length_squared() < 1:
            return p

def random_unit_vector():
    return unit_vector(random_in_unit_sphere())

def random_on_hemisphere(normal):
    on_unit_sphere = random_unit_vector()
    if dot(on_unit_sphere, normal) > 0.0:
        return on_unit_sphere
    else:
        return -on_unit_sphere

def reflect(v, n):
    return v - 2 * dot(v, n) * n

def refract(uv, n, etai_over_etat):
    cos_theta = min(dot(-uv, n), 1.0)
    r_out_perp = etai_over_etat * (uv + cos_theta * n)
    r_out_parallel = -np.sqrt(np.abs(1.0 - r_out_perp.length_squared())) * n
    return r_out_perp + r_out_parallel

def dot(u, v):
    return np.dot(u.e, v.e)

def cross(u, v):
    return Vec3(*np.cross(u.e, v.e))

def unit_vector(v):
    return v / v.length()

# point3 is an alias for vec3
Point3 = Vec3

# Overloaded Operators
def add(u, v):
    return Vec3(u.e[0] + v.e[0], u.e[1] + v.e[1], u.e[2] + v.e[2])

def sub(u, v):
    return Vec3(u.e[0] - v.e[0], u.e[1] - v.e[1], u.e[2] - v.e[2])

def mul(u, v):
    if isinstance(v, Vec3):
        return Vec3(u.e[0] * v.e[0], u.e[1] * v.e[1], u.e[2] * v.e[2])
    elif isinstance(v, (int, float)):
        return Vec3(u.e[0] * v, u.e[1] * v, u.e[2] * v)

def truediv(v, t):
    return Vec3(v.e[0] / t, v.e[1] / t, v.e[2] / t)


## 4. Rays, a simple camera, and background

### 4.1 The ray class

A direct translation of the C++ one. We add some useful methods to get/set a ray subset and get the number of rays.

In [9]:
"""
class Ray:
    def __init__(self, origin, direction):
        self.origin = origin
        self.direction = direction

    def at(self, t):
        return self.origin + self.direction*t
    
    def __getitem__(self, idx):
        return Ray(self.origin[idx], self.direction[idx])

    def __setitem__(self, idx, other):
        self.origin[idx] = other.origin
        self.direction[idx] = other.direction
    
    def __len__(self):
        return self.origin.x.size
"""
class Ray:
    def __init__(self, origin, direction):
        self.orig = origin
        self.dir = direction

    def origin(self):
        return self.orig

    def direction(self):
        return self.dir

    def at(self, t):
        return self.orig + t * self.dir

In [10]:
import numpy as np

# Helper function to write the color
def write_color(file, pixel_color):
    ir = int(255.999 * pixel_color[0])
    ig = int(255.999 * pixel_color[1])
    ib = int(255.999 * pixel_color[2])
    #print(f'{ir} {ig} {ib}')
    file.write(f"{ir} {ig} {ib}\n")

# Replace this with your ray_color function
"""
This method code assumes that `r.direction()` returns a `Vec3` object (or similar) with a `unit_vector` method, and that `color` is a function or class that takes three arguments and returns a color object. 
The `Vec3` object and the color object should both support multiplication and addition.
Please note that Python does not have a built-in `Vec3` or `color` class, so you would need to define these yourself or use a library that provides them.
"""
def ray_color(r):
    unit_direction = unit_vector(r.direction())
    a = 0.5 * (unit_direction.y + 1.0)
    #return  ( Color(1.0, 1.0, 1.0) * (1.0 - a) ) + ( Color(0.5, 0.7, 1.0) * a )
    return np.array([1.0, 0.0, 0.0])  # Dummy color (red)

# Camera and Image setup
aspect_ratio = 16.0 / 9.0
image_width = 400
image_height = max(int(image_width / aspect_ratio), 1)

# Camera parameters
focal_length = 1.0
viewport_height = 2.0
viewport_width = viewport_height * (image_width / image_height)
camera_center = Point3(0, 0, 0)

# Calculate vectors across the viewport edges
viewport_u = Vec3(viewport_width, 0, 0)
viewport_v = Vec3(0, -viewport_height, 0)

# Calculate the horizontal and vertical delta vectors from pixel to pixel
pixel_delta_u = viewport_u * (1 / image_width)
pixel_delta_v = viewport_v * ( 1/ image_height)

# Location of the upper left pixel
viewport_upper_left = camera_center - Vec3(0, 0, focal_length) - viewport_u / 2 - viewport_v / 2
pixel00_loc = viewport_upper_left + (pixel_delta_u + pixel_delta_v) * 0.5

# Render to a file
# Open a file to write the PPM image
with open("output2.ppm", "w") as file:
    # Write PPM header
    file.write("P3\n")
    file.write(f"{image_width} {image_height}\n")
    file.write("255\n")

    #print(f'P3\n{image_width} {image_height}\n255')
    for j in range(image_height):
        print(f'\rScanlines remaining: {image_height - j}', end='', flush=True)
        for i in range(image_width):
            pixel_center = pixel00_loc + (pixel_delta_u * i) + (pixel_delta_v * j )
            ray_direction = pixel_center - camera_center
            # Assuming `ray` is a function or class you have defined
            r = Ray(camera_center, ray_direction)
            pixel_color = ray_color(r)
            #print(pixel_color)
            write_color(file, pixel_color)
    #print('\rDone. \n')


TypeError: unsupported operand type(s) for *: 'method' and 'float'